In [10]:
from dotenv import load_dotenv
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [13]:
load_dotenv()

GPT_MODEL = 'gpt-3.5-turbo'

client = OpenAI()



In [15]:
response = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': 'Who won the world series in 2020?'},
    {'role': 'assistant', 'content': 'The Los Angeles Dodgers won the World Series in 2020.'},
    {'role': 'user', 'content': 'Where was it played?'}
  ]
)


ChatCompletion(id='chatcmpl-9hv9UvhuMOjnOM8S3OC4v80wcDYky', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas.', role='assistant', function_call=None, tool_calls=None))], created=1720254824, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))

In [25]:
response.choices[0].message.content

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

In [16]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print('Unable to generate ChatCompletion response')
        print(f'Exception: {e}')
        return e

In [26]:
class Agent:
    def __init__(self, model=GPT_MODEL):
        self.model = model
        self.memory = []
        self.tools = []
        
    def invoke(self, message):
        self.memory.append(
            {'role': 'user', 'content': message}
        )
        chat_response = chat_completion_request(
            messages=self.memory,
            tools=self.tools,
            model=self.model,
        )

        if chat_response.choices[0].finish_reason == 'stop':
            chat_response_message = chat_response.choices[0].message.content
            self.memory.append(
                {'role': 'assistant', 'content': chat_response_message}
            )
            return chat_response_message

        elif chat_response.choices[0].finish_reason == 'tool_calls':
            # to be implemented
            pass



In [27]:
agent = Agent()

agent.memory.append(
    {'role': 'system', 'content': 'You are a helpful assistant.'}
)

while True:
    user_input = input('User Message:')
    if user_input == 'exit':
        break
    response = agent.invoke(user_input)
    print('Assistant:', response)
    print()

Unable to generate ChatCompletion response
Exception: Error code: 400 - {'error': {'message': "Invalid 'tools': empty array. Expected an array with minimum length 1, but got an empty array instead.", 'type': 'invalid_request_error', 'param': 'tools', 'code': 'empty_array'}}


AttributeError: 'BadRequestError' object has no attribute 'choices'